In [2]:
import numpy as np
from os import listdir, mkdir, sep, path, walk
from os.path import join, exists, splitext
from scipy.misc import imread, imsave, imresize

In [3]:
from matplotlib import pyplot as plt

In [4]:
'''
Train or make inference the style transfer network
'''

from __future__ import print_function

import tensorflow as tf

IS_TRAINING = False

# for training
TRAINING_CONTENT_DIR = '../../_input/content'
TRAINING_STYLE_DIR = '../../_input/style'
ENCODER_WEIGHTS_PATH = '../../vgg19_normalised.npz'
LOGGING_PERIOD = 20

STYLE_WEIGHTS = [3.0]
CONTENT_WEIGHTS = [1.0]
MODEL_SAVE_PATHS = ['../../models/style_weight_2e0.ckpt']

# for inferring (stylize)
INFERRING_CONTENT_DIR = '../../_inference/content'
INFERRING_STYLE_DIR = '../../_inference/style'
OUTPUTS_DIR = '../../_inference/output'


In [5]:
def list_images(directory):
    images = []
    for file in listdir(directory):
        name = file.lower()
        if name.endswith('.png'):
            images.append(join(directory, file))
        elif name.endswith('.jpg'):
            images.append(join(directory, file))
        elif name.endswith('.jpeg'):
            images.append(join(directory, file))
    return images

In [6]:
content_imgs_path = list_images(INFERRING_CONTENT_DIR)

In [7]:
content_imgs_path

['../../_inference/content\\brad_pitt.jpg']

In [8]:
def get_images(paths, height=None, width=None):
    if isinstance(paths, str):
        paths = [paths]

    images = []
    for path in paths:
        image = imread(path, mode='RGB')

        if height is not None and width is not None:
            image = imresize(image, [height, width], interp='nearest')
        
        # Escape image with odd shapes (for training)
        height = int(image.shape[0] / 2) * 2
        width  = int(image.shape[1] / 2) * 2
        image  = imresize(image, [height, width], interp='nearest')

        images.append(image)

    images = np.stack(images, axis=0)

    return images

In [9]:
img = get_images(content_imgs_path)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  from ipykernel import kernelapp as app


In [10]:
img.shape

(1, 1330, 1054, 3)

In [26]:
# plt.imshow(img)